In [1]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import csv
import numpy as np
import pandas as pd
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv('clean_data.csv')
df

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.75245,-79.32991
1,1,M4A,North York,Victoria Village,43.73057,-79.31306
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
5,5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.66263,-79.52831
6,6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,7,M3B,North York,Don Mills,43.74923,-79.36186
8,8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


# Extracting Boroughs that has toronto attached to their name

In [3]:
df_toronto_only = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_only.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,15,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,19,M4E,East Toronto,The Beaches,43.67709,-79.29547


In [9]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto are 43.6534817, -79.3839347.


In [12]:
import folium
map_toronto_only = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_only['Latitude'], 
        df_toronto_only['Longitude'], 
        df_toronto_only['Borough'], 
        df_toronto_only['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_only)  

map_toronto_only

In [16]:
neighborhood_name = df_toronto_only.loc[0, 'Neighborhood']
neighborhood_name


'Regent Park, Harbourfront'

In [17]:
CLIENT_ID= 'xxxxxxxxxxxxxxxxxxxx'
CLIENT_SECRET= 'xxxxxxxxxxxxxxxxx'
ACCESS_TOKEN= 'xxxxxxxxxxxxxxxxxxx'
VERSION='20180605'

In [26]:
url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [27]:
radius = 500
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            ACCESS_TOKEN,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
import requests
toronto_only_venues = getNearbyVenues(names=df_toronto_only['Neighborhood'],
                                   latitudes=df_toronto_only['Latitude'],
                                   longitudes=df_toronto_only['Longitude']
                                  )


In [30]:
toronto_only_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,"Regent Park, Harbourfront",43.65512,-79.36264,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Regent Park, Harbourfront",43.65512,-79.36264,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,Japango,43.655268,-79.385165,Sushi Restaurant
4,"Regent Park, Harbourfront",43.65512,-79.36264,Indigo,43.653515,-79.380696,Bookstore


In [31]:
toronto_only_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [32]:
print('There are {} uniques categories.'.format(len(toronto_only_venues['Venue Category'].unique())))

There are 61 uniques categories.


#  Analyze Each Neighborhood

In [33]:
toronto_onehot = pd.get_dummies(toronto_only_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_only_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Women's Store,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Distribution Center,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Garden,Gastropub,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lingerie Store,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Music Venue,Neighborhood,New American Restaurant,Office,Opera House,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salon / Barbershop,Seafood Restaurant,Shopping Mall,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
toronto_onehot.shape

(3900, 61)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Women's Store,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Distribution Center,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Garden,Gastropub,Greek Restaurant,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Lingerie Store,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Music Venue,New American Restaurant,Office,Opera House,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salon / Barbershop,Seafood Restaurant,Shopping Mall,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant
0,Berczy Park,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
1,"Brockton, Parkdale Village, Exhibition Place",0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
2,"Business reply mail Processing Centre, South C...",0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
4,Central Bay Street,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
5,Christie,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
6,Church and Wellesley,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
7,"Commerce Court, Victoria Hotel",0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
8,Davisville,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01,0.02,0.02,0.03,0.02,0.01,0.02,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.03,0.01,0.01,0.01,0.02,0.01,0.01,0.03,0.01,0.02,0.01,0.01
9,Davisville North,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.02,0.02,0.1,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.01,0.01

##### Let's confirm the new size

In [36]:
toronto_grouped.shape

(39, 61)

#### Let's print each neighborhood along with the top 5 most common venues

In [38]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0       Clothing Store  0.10
1                Hotel  0.03
2  American Restaurant  0.03
3   Seafood Restaurant  0.03
4       Cosmetics Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0       Clothing Store  0.10
1                Hotel  0.03
2  American Restaurant  0.03
3   Seafood Restaurant  0.03
4       Cosmetics Shop  0.03


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0       Clothing Store  0.10
1                Hotel  0.03
2  American Restaurant  0.03
3   Seafood Restaurant  0.03
4       Cosmetics Shop  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Clothing Store  0.10
1                Hotel  0.03
2  American Restaurant  0.03
3   Seafood Restaurant  0.03
4       Cosmetics Shop  0.03


----Central Bay

### Let's put that into a panda

First, let's write a function to sort the venues in descending order.

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Clothing Store,American Restaurant,Tea Room,Hotel,Cosmetics Shop,Seafood Restaurant,Coffee Shop,Gym / Fitness Center,Café,Garden
1,"Brockton, Parkdale Village, Exhibition Place",Clothing Store,American Restaurant,Tea Room,Hotel,Cosmetics Shop,Seafood Restaurant,Coffee Shop,Gym / Fitness Center,Café,Garden
2,"Business reply mail Processing Centre, South C...",Clothing Store,American Restaurant,Tea Room,Hotel,Cosmetics Shop,Seafood Restaurant,Coffee Shop,Gym / Fitness Center,Café,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Clothing Store,American Restaurant,Tea Room,Hotel,Cosmetics Shop,Seafood Restaurant,Coffee Shop,Gym / Fitness Center,Café,Garden
4,Central Bay Street,Clothing Store,American Restaurant,Tea Room,Hotel,Cosmetics Shop,Seafood Restaurant,Coffee Shop,Gym / Fitness Center,Café,Garden


# Cluster Neighborhoods

In [47]:
# set number of clusters
kclusters = 1

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_only

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

ValueError: cannot insert Cluster Labels, already exists